In [ ]:
import subprocess
import pandas as pd
label_df ="WK_digits_rememberd.csv"
command_label = f"dx download file-Gx8pB38Jk6kP63J7Ykbb2gPF --output /opt/notebooks/{label_df} --overwrite"

label_df_alc = "alc_multi_label.csv"
command_label_alc = f"dx download file-GxJYyQjJk6kPfqG079ZBPYg6 --output /opt/notebooks/{label_df_alc} --overwrite"

multi_label_df = "merged_multitarget_df_with_demographics.csv"
command_multi_label = f"dx download file-GxK2v3jJk6k9FppfBZqGQj1v --output /opt/notebooks/{multi_label_df} --overwrite"

In [ ]:
mri_table = "aparc.thickness_aparc.volume_aseg.volume.csv"
command = f"dx download file-GxG4P20Jk6k9kG9pv7Bxx2pk --output '/opt/notebooks/{mri_table}' --overwrite"
subprocess.run(command, shell=True, check=True)

In [ ]:
subprocess.run(command_label, shell=True, check=True)
subprocess.run(command_label_alc, shell=True, check=True)
subprocess.run(command_multi_label, shell=True, check=True)

In [ ]:
label_wm = pd.read_csv(f"/opt/notebooks/{label_df}")
label_df_alc = pd.read_csv(f"/opt/notebooks/{label_df_alc}")
multi_label_df = pd.read_csv(f"/opt/notebooks/{multi_label_df}")
mri_df = pd.read_csv(f"/opt/notebooks/{mri_table}")
mri_df["ID"] = mri_df["ID"].str.replace("sub-", "")

In [ ]:
mri_df.to_csv(f"/opt/notebooks/{mri_table}", index=False)

In [ ]:
high_wm_range = range(9, 13)  # Scores 9 to 12 inclusive
low_wm_range = range(2, 4)    # Scores 2 to 5 inclusive
def classify_wm_capacity(score):
    if score in high_wm_range:
        return 1  # High WM capacity
    elif score in low_wm_range:
        return 0  # Low WM capacity
    else:
        return None  # Exclude other scores

# Apply classification
label_wm['Good_WM_Memory'] = label_wm["digits_remembered"].apply(classify_wm_capacity)
df_wm = label_wm.dropna(subset=['Good_WM_Memory'])

In [ ]:
label_wm.head(2)
label_wm["label_Bad_WM_Memory"] = 1 - label_wm["Good_WM_Memory"]

In [ ]:
label_wm["digits_remembered"].value_counts()

In [ ]:
label_wm["label_Bad_WM_Memory"].value_counts()

In [ ]:
#merge wm to multi_label_df
merged_df = pd.merge(multi_label_df, label_wm, on="ID", how="left")
merged_df = merged_df.drop(columns=["digits_remembered", "Good_WM_Memory"])

In [ ]:
merged_df.to_csv("/opt/notebooks/merged_multitarget_df_with_demographics_wm.csv", index=False)

In [ ]:
cols_to_drop = [
    "label_HRD_sex_balanced",
    "label_HRD_balanced",
    "label_HRD",
    "label_HRD_Binge>1_no_ol_balanced",
    "label_HRD_Binge>1_no_ol_sex_balanced",
    "label_HRD_Binge>1_no_ol_balanced",
    "label_HRD_Binge>1_no_ol"
]
merged_df = merged_df.drop(cols_to_drop, axis=1)


In [ ]:
merged_df["ID"] = merged_df["ID"].astype(str)

In [ ]:
merged_df = pd.merge(merged_df, mri_df, on="ID", how="inner")
#define a sick label. IF in the mulit_label df a column with the name "label_*"" is 1 hte sick column is 1 else 0
merged_df['label_sick'] = (merged_df.filter(like='label_') == 1).any(axis=1).astype(int)


In [ ]:
merged_df["label_sick"].value_counts()

In [ ]:
#drop the columns from mri_df but the "ID"
cols_to_drop = [col for col in mri_df.columns if col != "ID"]
merged_df.drop(cols_to_drop, axis=1, inplace=True)

In [ ]:
for col in merged_df.columns:
    print(col)

In [ ]:
merged_df.to_csv("/opt/notebooks/merged_multitarget_df_with_demographics_wm.csv", index=False)

In [ ]:
def match_participant(target_row, df_candidates, relax_order, used_ids,thresholds):
    df_candidates = df_candidates[~df_candidates['ID'].isin(used_ids)].copy()
    for relax_level in relax_order:
        filtered = df_candidates.copy()
        for criterion, match_exact in relax_level.items():
            if criterion in ['assessment_centre']:
                if match_exact:
                    filtered = filtered[filtered[criterion] == target_row[criterion]]
            elif criterion in ['sex']:
                if match_exact:
                    #find opposite sex
                    filtered = filtered[filtered[criterion] != target_row[criterion]]
            else:
                if match_exact:
                    diff = abs(filtered[criterion] - target_row[criterion])
                    filtered = filtered[diff <= thresholds[criterion]]
            if filtered.empty:
                break
        if not filtered.empty:
            return filtered.iloc[0]
    print("NO candidate found")
    return None
relax_order = [
    {'assessment_centre': True, 'deprivation_index': True, 'bmi': True, 'age_at_assessment': True, 'sex': True},
    {'assessment_centre': False, 'deprivation_index': True, 'bmi': True, 'age_at_assessment': True, 'sex': True},
    {'assessment_centre': False, 'deprivation_index': False, 'bmi': True, 'age_at_assessment': True, 'sex': True},
    {'assessment_centre': False, 'deprivation_index': False, 'bmi': False, 'age_at_assessment': True, 'sex': True},
    {'assessment_centre': False, 'deprivation_index': False, 'bmi': False, 'age_at_assessment': False, 'sex': True},
]
thresholds = {
    'age_at_assessment': 2,
    'bmi': 3,
    'deprivation_index': 1
}


In [ ]:
from joblib import Parallel, delayed
import os
def match_for_label(label, merged_df, relax_order, thresholds):
    positive_group = merged_df[merged_df[label] == 1]
    negative_group = merged_df[merged_df[label] == 0]

    matches = []
    used_ids = set() 
    for _, target_row in positive_group.iterrows():
        matched = match_participant(target_row, negative_group, relax_order, used_ids, thresholds)
        if matched is not None:
            matches.append(matched)
            used_ids.add(matched['ID'])
    matched_df = pd.DataFrame(matches)
    return label, matched_df, positive_group

label_cols = ["label_sick"]
results = Parallel(n_jobs=-1)(delayed(match_for_label)(label, merged_df, relax_order, thresholds) for label in label_cols)

In [ ]:
import time
logs_path = "project-GqzxkVQJ34g6ygFJ4ZbvqBYF:/Esra/00_CLIP/04_trainings_data/02_normative/"
for label, matched_df, positive_group in results:
    #concat the the positive and negative matched participants
    final_df = pd.concat([positive_group, matched_df])
    final_df["ID"] =final_df["ID"].astype(int).astype(str)
    final_df.to_csv(f"/opt/notebooks/Matched_validation_sick_healthy_less_strict.csv", index=False)
    command = f"dx upload /opt/notebooks/Matched_validation_sick_healthy_less_strict.csv --path {logs_path}"
    print(command)
    subprocess.run(command, shell=True, check=True)

In [ ]:
import pandas as pd
logs_path = "project-GqzxkVQJ34g6ygFJ4ZbvqBYF:/Esra/00_CLIP/04_trainings_data/02_normative/"

In [ ]:
final_df = pd.read_csv("/opt/notebooks/Matched_validation_sick_healthy_less_strict.csv")

In [ ]:
final_df["ID"] = final_df["ID"].astype(str)
merged_df["ID"] = merged_df["ID"].astype(str)

# Get the unique IDs from each DataFrame
final_df_ids = set(final_df["ID"])
merged_df_ids = set(merged_df["ID"])

# Find IDs present in merged_df but not in final_df
not_matched_ids = merged_df_ids - final_df_ids

# Filter the DataFrame to get rows with unmatched IDs
not_matched_df = merged_df[merged_df["ID"].isin(not_matched_ids)]

In [ ]:
not_matched_df.to_csv(f"/opt/notebooks/healthy_subjects_train_less_strict.csv", index=False)
command = f"dx upload /opt/notebooks/healthy_subjects_train_less_strict.csv --path {logs_path}"
print(command)
subprocess.run(command, shell=True, check=True)

In [ ]:
label_cols = ["label_Bad_WM_Memory"]
results = Parallel(n_jobs=-1)(delayed(match_for_label)(label, final_df, relax_order, thresholds) for label in label_cols)

In [ ]:
for label, matched_df, positive_group in results:
    #concat the the positive and negative matched participants
    final_df_2 = pd.concat([positive_group, matched_df])

In [ ]:
final_df["label_Bad_WM_Memory"].value_counts()

In [ ]:
postive = df[df["label_Bad_WM_Memory"] == 1]
negative = df[df["label_Bad_WM_Memory"] == 0]

In [ ]:
import matplotlib.pyplot as plt
#check the distribution of the matched participants for age_at_assessment bmi and so on 
col_to_check = ["age_at_assessment", "bmi", "deprivation_index", "sex", "assessment_centre"]
for col in col_to_check:
    print(col)
    print(postive[col].hist())
    print(negative[col].hist())
    print(postive[col].describe())
    print(negative[col].describe())
    print("")
    plt.show()